In [ ]:
%%capture
pip install duckdb ipywidgets ipyleaflet pandas

In [ ]:
from ipyleaflet import Map, Marker, Popup, LayerGroup, MarkerCluster, WidgetControl, AwesomeIcon
from ipywidgets import HTML, IntSlider, Button, Layout
import pandas as pd

In [ ]:
import duckdb
duckdb = duckdb.connect('/duckdb/dev.duckdb', config={})
duckdb.query("INSTALL SPATIAL; LOAD SPATIAL")

In [ ]:
station_utilization_query = """
with events as (
    select *
    , e.event_at::DATE as date
    , extract(MONTH from e.event_at) as month
    from divvy_station_events e
),
grouped_events as (
select s.name, s.lat, s.lon, s.active, count(*) as num_events
from divvy_stations s

left join events e on s.station_id = e.station_id

where 1=1
and s.active = true

group by 1,2,3,4

having 1=1

-- a station's data and status is considered reliable if it has events spanning multiple months
-- this filters stations that may have been inactive, are new, etc
and count(distinct month) >= 3

order by num_events desc
)

select * from (select * from grouped_events order by num_events desc limit $limit)
union all
select * from (select * from grouped_events order by num_events asc limit $limit)
order by num_events desc

"""

In [ ]:
m = Map()

def reset_zoom(self):
    m.center = (41.8, -87.5)
    m.zoom = 10


reset_zoom(None)

map_markers = LayerGroup()
m.add(map_markers)

button = Button(icon="home", tooltip="Reset Zoom", layout=Layout(width='35px'))
button.on_click(reset_zoom)
m.add(WidgetControl(widget=button, position='topleft'))

slider = IntSlider(value=10, min=1, max=25, step=1, description='Limit:', style=dict(description_width='100px'), continuous_update=False)
m.add(WidgetControl(widget=slider, position='topright'))

display(m)

In [ ]:
def draw_chart(change):
    duckdb.query(station_utilization_query, params={"limit":change["new"]}).df().plot.bar(x="name", y="num_events", title="Station Utilization", logy=True)
    
draw_chart({'new': slider.value})
slider.observe(draw_chart, names='value')

In [ ]:
def draw_markers(change):
    map_markers.clear_layers()

    rows = duckdb.query(station_utilization_query, params={"limit":change['new']}).fetchall()
    
    stations = {}
    
    for row in rows:
        message = HTML()
        message.value = f"{row[0]}<br>{row[4]} events"
        map_markers.add(Marker(location=(row[1], row[2]), draggable=False, popup=message, icon=AwesomeIcon(name='bicycle', marker_color='lightblue' if row[4] < 1000 else 'blue')))
        
draw_markers({'new': slider.value})

slider.observe(draw_markers, names='value')